In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - 0_make_merger_data.ipynb
# AUTHOR - James Lane
# PROJECT - tng-dfs
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Make some convenient data files for the merger & DF analysis
'''

__author__ = "James Lane"

In [ ]:
# %load ../../src/nb_modules/nb_imports.txt
### Imports

## Basic
import numpy as np
import sys, os
import h5py, dill as pickle
import pdb, copy, glob, time, subprocess, warnings, multiprocessing

## Matplotlib
import matplotlib as mpl
from matplotlib import pyplot as plt

## Astropy
from astropy import units as apu

## Analysis
import scipy.stats
import scipy.interpolate
import emcee
import corner

## Project-specific
src_path = 'src/'
while True:
    if os.path.exists(src_path): break
    if os.path.realpath(src_path).split('/')[-1] in ['tng-dfs','/']:
            raise FileNotFoundError('Failed to find src/ directory.')
    src_path = os.path.join('..',src_path)
sys.path.insert(0,src_path)
from tng_dfs import cutout as pcutout
from tng_dfs import densprofile as pdens
from tng_dfs import fitting as pfit
from tng_dfs import io as pio
from tng_dfs import kinematics as pkin
from tng_dfs import plot as pplot
from tng_dfs import tree as ptree
from tng_dfs import util as putil

### Notebook setup

%matplotlib inline
plt.style.use(os.path.join(src_path,'mpl/project.mplstyle')) # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Keywords, loading, pathing

In [ ]:
# %load ../../src/nb_modules/nb_setup.txt
# Keywords
cdict = putil.load_config_to_dict()
keywords = ['DATA_DIR','MW_ANALOG_DIR','FIG_DIR_BASE','FITTING_DIR_BASE',
            'RO','VO','ZO','LITTLE_H','MW_MASS_RANGE']
data_dir,mw_analog_dir,fig_dir_base,fitting_dir_base,ro,vo,zo,h,\
    mw_mass_range = putil.parse_config_dict(cdict,keywords)

# MW Analog 
mwsubs,mwsubs_vars = putil.prepare_mwsubs(mw_analog_dir,h=h,
    mw_mass_range=mw_mass_range,return_vars=True,force_mwsubs=False,
    bulge_disk_fraction_cuts=True)

# Load tree data
tree_primary_filename = os.path.join(mw_analog_dir,
    'major_mergers/tree_primaries.pkl')
with open(tree_primary_filename,'rb') as handle: 
    tree_primaries = pickle.load(handle)
tree_major_mergers_filename = os.path.join(mw_analog_dir,
    'major_mergers/tree_major_mergers.pkl')
with open(tree_major_mergers_filename,'rb') as handle:
    tree_major_mergers = pickle.load(handle)
n_mw = len(tree_primaries)

### Version information

In [ ]:
# Versions for the analysis
analysis_version = 'v1.1'

# Versions for density profiles
stellar_halo_density_version = 'poisson_twopower_softening'
stellar_halo_rotation_dftype = 'tanh_rotation'
stellar_halo_rotation_version = 'asymmetry_fit'

### Create a large structured array with all the information about the mergers

In [ ]:
# Keywords and paths
verbose = True
force_compute = False
analysis_dir = os.path.join(mw_analog_dir,'analysis',analysis_version)
os.makedirs(analysis_dir,exist_ok=True)
merger_data_filename = os.path.join(analysis_dir,'merger_data.npy')
dens_fitting_dir = os.path.join(fitting_dir_base,'density_profile')
df_fitting_dir = os.path.join(fitting_dir_base,'distribution_function')

# Stellar halo density information
stellar_halo_density_ncut = 500
stellar_halo_rotation_ncut = 500

# DF information
df_type = ['constant_beta','osipkov_merritt','osipkov_merritt_2_combination']
fit_version = ['anisotropy_params_softening','anisotropy_params_softening',
               'ra_N10_01_to_300_softening']
df_ncut = 500

# Densfunc
stellar_halo_densfunc = pdens.TwoPowerSpherical()

# Array to hold merger data
merger_data = []

for i in range(n_mw):

    # Get the primary
    primary = tree_primaries[i]
    z0_sid = primary.subfind_id[0]
    major_mergers = primary.tree_major_mergers
    n_major = primary.n_major_mergers
    primary_filename = primary.get_cutout_filename(mw_analog_dir,
    snapnum=primary.snapnum[0])

    # Get DM and star particle IDs and masses for the primary
    co = pcutout.TNGCutout(primary_filename)
    dmpid = co.get_property('dm','ParticleIDs')
    dmass = co.get_masses('dm').to_value(apu.Msun)
    spid = co.get_property('stars','ParticleIDs')
    smass = co.get_masses('stars').to_value(apu.Msun)

    # Loop over major mergers
    for j in range(n_major):
        if verbose:
            print(f'Processing major merger {j+1}/{n_major} '
                  f'for MW analog {i+1}/{n_mw}', end='\r')
        
        # Get the merger
        major_merger = primary.tree_major_mergers[j]
        major_acc_sid = major_merger.subfind_id[0]
        major_mlpid = major_merger.secondary_mlpid
        merger_number = j+1

        # Get the unique PIDs for the merger
        dmupid = major_merger.get_unique_particle_ids('dm',data_dir=data_dir)
        supid = major_merger.get_unique_particle_ids('stars',data_dir=data_dir)
        dmindx = np.isin(dmpid, dmupid)
        sindx = np.isin(spid, supid)
        star_mass = np.sum(smass[sindx])
        dm_mass = np.sum(dmass[dmindx])

        # Information about the merger
        star_mass_ratio = major_merger.star_mass_ratio
        star_mass_ratio_snapnum = major_merger.star_mass_ratio_snapnum
        dm_mass_ratio = major_merger.dm_mass_ratio
        dm_mass_ratio_snapnum = major_merger.dm_mass_ratio_snapnum
        merger_snapnum = major_merger.merger_snapnum
        merger_redshift = putil.snapshot_to_redshift(merger_snapnum)

        # Get the stellar halo density profile (denspot for the DF)
        stellar_halo_density_filename = os.path.join(dens_fitting_dir,
            'stellar_halo/',stellar_halo_density_version,str(z0_sid),
            'merger_'+str(j+1)+'/', 'sampler.pkl')
        denspot = pfit.construct_pot_from_fit(
            stellar_halo_density_filename, stellar_halo_densfunc, 
            stellar_halo_density_ncut, ro=ro, vo=vo)
        alpha = denspot.alpha
        beta = denspot.beta
        a = denspot.a*ro

        # Get the stellar halo rotation kernel
        stellar_halo_rotation_filename = os.path.join(df_fitting_dir,
            stellar_halo_rotation_dftype,stellar_halo_rotation_version,
            str(z0_sid),'merger_'+str(j+1)+'/', 'sampler.pkl')
        krot, chi = \
            pio.median_params_from_emcee_sampler(stellar_halo_rotation_filename,
                ncut=stellar_halo_rotation_ncut)

        # Load the constant beta information
        anisotropy_dir = os.path.join(df_fitting_dir, df_type[0],
            fit_version[0],str(z0_sid),'merger_'+str(j+1))
        anisotropy_filename = os.path.join(anisotropy_dir,'sampler.pkl')
        assert os.path.exists(anisotropy_filename)
        with open(anisotropy_filename,'rb') as handle:
            beta_sampler = pickle.load(handle)
        anisotropy_samples = beta_sampler.get_chain(discard=df_ncut,flat=True)
        anisotropy = np.median(anisotropy_samples,axis=0)[0]

        # Load the Osipkov-Merritt information
        om_dir = os.path.join(df_fitting_dir, df_type[1],
            fit_version[1],str(z0_sid),'merger_'+str(j+1))
        om_filename = os.path.join(om_dir,'sampler.pkl')
        assert os.path.exists(om_filename)
        with open(om_filename,'rb') as handle:
            om_sampler = pickle.load(handle)
        ra_samples = om_sampler.get_chain(discard=df_ncut, flat=True)
        ra = np.median(ra_samples,axis=0)[0]

        # Load the combined Osipkov-Merritt information
        om2_dir = os.path.join(df_fitting_dir, df_type[2],
            fit_version[2],str(z0_sid),'merger_'+str(j+1))
        om2_filename = os.path.join(om2_dir,'sampler.pkl')
        assert os.path.exists(om2_filename)
        with open(om2_filename,'rb') as handle:
            om2_sampler = pickle.load(handle)
        om2_samples = om2_sampler.get_chain(discard=df_ncut, flat=True)
        ra1,ra2,kom = np.median(om2_samples,axis=0)

        # Construct the tuple of merger data
        merger_data.append((
            z0_sid,
            major_acc_sid,
            major_mlpid,
            merger_number,
            star_mass_ratio,
            star_mass_ratio_snapnum,
            dm_mass_ratio,
            dm_mass_ratio_snapnum,
            merger_snapnum,
            merger_redshift,
            star_mass,
            dm_mass,
            alpha,
            beta,
            krot,
            chi,
            a,
            anisotropy,
            ra,
            ra1,
            ra2,
            kom
            ))
        
# Create the structured array dtype
merger_dtype = np.dtype([
    ('z0_sid', np.int64),
    ('major_acc_sid', np.int64),
    ('major_mlpid', np.int64),
    ('merger_number', np.int64),
    ('star_mass_ratio', np.float64),
    ('star_mass_ratio_snapnum', np.int64),
    ('dm_mass_ratio', np.float64),
    ('dm_mass_ratio_snapnum', np.int64),
    ('merger_snapnum', np.int64),
    ('merger_redshift', np.float64),
    ('star_mass', np.float64),
    ('dm_mass', np.float64),
    ('alpha', np.float64),
    ('beta', np.float64),
    ('krot', np.float64),
    ('chi', np.float64),
    ('a', np.float64),
    ('anisotropy', np.float64),
    ('ra', np.float64),
    ('ra1', np.float64),
    ('ra2', np.float64),
    ('kom', np.float64)
])

merger_data = np.array(merger_data,dtype=merger_dtype)
np.save(merger_data_filename,merger_data)